In [3]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.models import model_from_json

In [4]:
def preprocessing(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)
    image = image/255
    return image


features = []
targets = []
for i in ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]:
    collectionImageNames = os.listdir("D:/another data" + "/" + str(i))
    for j in collectionImageNames:
        img = cv2.imread("D:/another data" + "/" + str(i) + "/" + j)
        features.append(img)
        if i == "angry":
            targets.append(0)
        elif i == "disgust":
            targets.append(1)
        elif i == "fear":
            targets.append(2)
        elif i == "happy":
            targets.append(3)
        elif i == "neutral":
            targets.append(4)
        elif i == "sad":
            targets.append(5)
        elif i == "surprise":
            targets.append(6)
    print("Loading in folder", i)


Loading in folder angry
Loading in folder disgust
Loading in folder fear
Loading in folder happy
Loading in folder neutral
Loading in folder sad
Loading in folder surprise


In [5]:
features = np.array(features)
targets = np.array(targets)
train_features, test_features, train_target, test_target = train_test_split(features, targets, test_size=0.2)



In [8]:
features.shape

(28821, 48, 48, 3)

In [9]:
targets.shape

(28821,)

In [4]:
train_features = np.array(list(map(preprocessing, train_features)))
test_features = np.array(list(map(preprocessing, test_features)))


In [5]:
print(train_features.shape)
print(test_features.shape)

(23056, 48, 48)
(5765, 48, 48)


In [6]:
train_features = train_features.reshape(23056, 48, 48, 1)
test_features = test_features.reshape(5765, 48, 48, 1)


In [7]:
print(train_features.shape)
print(test_features.shape)

(23056, 48, 48, 1)
(5765, 48, 48, 1)


In [8]:
DataGen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1,
                             shear_range=0.1, zoom_range=0.2)
DataGen.fit(train_features)
batches = DataGen.flow(train_features, train_target, batch_size=20)
train_target = to_categorical(train_target)


In [13]:
# step 1------ Specify the architecture
model = Sequential()
model.add(Conv2D(60, (3, 3), activation="relu", input_shape=(48, 48, 1)))
model.add(Conv2D(60, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(30, (3, 3), activation="relu"))
model.add(Conv2D(30, (3, 3), activation="relu"))
model.add(Conv2D(30, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(200, activation="relu"))
model.add(Dense(200, activation="relu"))
model.add(Dense(7, activation="softmax"))


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 46, 46, 60)        600       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 44, 44, 60)        32460     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 60)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 30)        16230     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 18, 18, 30)        8130      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 30)        8130      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 8, 8, 30)         

In [15]:
# Step 2----- Compile the model
model.compile(Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])


In [16]:
# Step 3-----Train the model
model.fit(DataGen.flow(train_features, train_target, batch_size=20), epochs=20)


Epoch 1/20
1153/1153 [==============================] - 78s 67ms/step - loss: 1.8233 - accuracy: 0.2475
Epoch 2/20
1153/1153 [==============================] - 74s 64ms/step - loss: 1.7422 - accuracy: 0.2826
Epoch 3/20
1153/1153 [==============================] - 73s 63ms/step - loss: 1.6858 - accuracy: 0.3257
Epoch 4/20
1153/1153 [==============================] - 73s 63ms/step - loss: 1.6059 - accuracy: 0.3653
Epoch 5/20
1153/1153 [==============================] - 73s 63ms/step - loss: 1.5560 - accuracy: 0.3919
Epoch 6/20
1153/1153 [==============================] - 73s 64ms/step - loss: 1.5133 - accuracy: 0.4104
Epoch 7/20
1153/1153 [==============================] - 73s 63ms/step - loss: 1.4968 - accuracy: 0.4167
Epoch 8/20
1153/1153 [==============================] - 74s 64ms/step - loss: 1.4656 - accuracy: 0.4361
Epoch 9/20
1153/1153 [==============================] - 73s 64ms/step - loss: 1.4524 - accuracy: 0.4356
Epoch 10/20
1153/1153 [==============================] - 73s 64m

In [17]:
# saving the model architecture in the json file
Model_In_Json = model.to_json()
abc = open("emotion_detection_model.json", "w")
abc.write(Model_In_Json)
abc.close()
# saving the model weights in the h5 file
model.save_weights("emotion_detection_weights.h5")


In [18]:
# Reading(loading..) the json file to use it in the model
abc = open("emotion_detection_model.json", "r")
loaded_data = abc.read()
loaded_model = model_from_json(loaded_data)
loaded_model.load_weights("emotion_detection_weights.h5")


In [ ]:
# Step 4------Test the model for the predictions
for i in range(10):
    predictions = loaded_model.predict(test_features)
    print(predictions[i])
    print(np.max(predictions[i]))

In [ ]:
print(test_target[1])

In [47]:
import cv2
import numpy as np
from keras.models import model_from_json

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


def getclassname(classno):
    if classno == 0:
        return "angry"
    elif classno == 1:
        return "disgust"
    elif classno == 2:
        return "fear"
    elif classno == 3:
        return "happy"
    elif classno == 4:
        return "neutral"
    elif classno == 5:
        return "sad"
    elif classno == 6:
        return "surprise"


def preprocessing(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image/255
    return image


abc = open("emotion_detection_model.json", "r")
loaded_data = abc.read()
loaded_model = model_from_json(loaded_data)
loaded_model.load_weights("emotion_detection_weights.h5")

capt = cv2.VideoCapture(0)
capt.set(3, 640)
capt.set(4, 480)
capt.set(10, 180)
while True:
    message, image = capt.read()
    image_arr = np.asarray(image)
    image_arr = cv2.resize(image_arr, (48, 48))
    image_arr = preprocessing(image_arr)
    image_arr = image_arr.reshape(1, 48, 48, 1)
    predictions = loaded_model.predict(image_arr)
    Neuron_index = loaded_model.predict(image_arr)
    cv2.putText(image, "Class: ", (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.putText(image, "Probability: ", (20, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    probability_value = np.amax(predictions)
    if probability_value > 0.50:
        cv2.putText(image, getclassname(Neuron_index), (120, 35), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(image, str(int(probability_value*100)) + "%", (200, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("Model Prediction", image)
    ascii_value = cv2.waitKey(1)
    if ascii_value == ord("q"):
        cv2.destroyAllWindows()
        break


error: OpenCV(4.5.3) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src data type = 17 is not supported
>  - Expected Ptr<cv::UMat> for argument 'src'


In [1]:
print(Neuron_index)

NameError: name 'Neuron_index' is not defined